# 05. Supervised Fine-Tuning (SFT)
## Synthetic Instruction Tuner - Week 3 Day 1-3

This notebook performs supervised fine-tuning on the filtered instruction dataset:
1. Load base model (Llama-3.2-3B/Mistral-7B/Qwen2.5-3B)
2. Prepare SFT training data
3. Configure LoRA for parameter-efficient fine-tuning
4. Train with TRL's SFTTrainer
5. Evaluate and save the fine-tuned model

**Training settings**:
- LoRA: r=8, alpha=16
- Epochs: 3
- Learning rate: 2e-4
- Batch size: 4 (with gradient accumulation)

**Expected runtime**: 6-10 hours on Colab T4

**Tip**: Monitor training loss and save checkpoints regularly

## 1. Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Project path
PROJECT_ROOT = "/content/drive/MyDrive/synthetic-instruction-tuner"

In [ ]:
# Load configuration
import json

with open(f"{PROJECT_ROOT}/config.json", 'r') as f:
    config = json.load(f)

print("Configuration loaded!")

In [ ]:
# Install libraries with latest compatible versions (avoid dependency conflicts)
!pip install -q --upgrade transformers>=4.41.0 peft>=0.7.0 trl>=0.7.4 datasets>=2.16.0 accelerate>=0.25.0 bitsandbytes>=0.41.3

print("✅ Libraries installed successfully!")

In [ ]:
import torch
import json
import os
import time
from datetime import datetime
from datasets import Dataset
import gc

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Efficiency Metrics Tracker for Method Comparison
class EfficiencyTracker:
    """Track efficiency metrics for adaptation method comparison."""
    
    def __init__(self, method_name: str):
        self.method_name = method_name
        self.metrics = {
            "method": method_name,
            "memory_allocated_gb": [],
            "memory_reserved_gb": [],
            "training_time_seconds": 0,
            "trainable_params": 0,
            "total_params": 0,
            "trainable_ratio": 0,
            "inference_tokens_per_sec": 0,
        }
        self.start_time = None
    
    def log_memory(self):
        """Log current GPU memory usage."""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1e9
            reserved = torch.cuda.memory_reserved() / 1e9
            self.metrics["memory_allocated_gb"].append(allocated)
            self.metrics["memory_reserved_gb"].append(reserved)
            return {"allocated": allocated, "reserved": reserved}
        return None
    
    def start_training(self):
        """Start timing training."""
        self.start_time = time.time()
        self.log_memory()
    
    def end_training(self):
        """End timing training."""
        if self.start_time:
            self.metrics["training_time_seconds"] = time.time() - self.start_time
        self.log_memory()
    
    def log_params(self, model):
        """Log parameter counts."""
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total = sum(p.numel() for p in model.parameters())
        self.metrics["trainable_params"] = trainable
        self.metrics["total_params"] = total
        self.metrics["trainable_ratio"] = trainable / total if total > 0 else 0
    
    def log_inference_speed(self, tokens_generated: int, time_taken: float):
        """Log inference speed."""
        self.metrics["inference_tokens_per_sec"] = tokens_generated / time_taken if time_taken > 0 else 0
    
    def get_summary(self):
        """Get summary metrics."""
        summary = {
            "method": self.method_name,
            "trainable_params": self.metrics["trainable_params"],
            "total_params": self.metrics["total_params"],
            "trainable_ratio_percent": self.metrics["trainable_ratio"] * 100,
            "peak_memory_gb": max(self.metrics["memory_allocated_gb"]) if self.metrics["memory_allocated_gb"] else 0,
            "training_time_hours": self.metrics["training_time_seconds"] / 3600,
            "inference_tokens_per_sec": self.metrics["inference_tokens_per_sec"],
        }
        return summary
    
    def save(self, path: str):
        """Save metrics to JSON."""
        with open(path, 'w') as f:
            json.dump(self.get_summary(), f, indent=2)
        print(f"Metrics saved to {path}")

# Initialize tracker for LoRA
tracker = EfficiencyTracker("lora_r8")
print("Efficiency tracker initialized for LoRA!")

## 2. Load Training Data

In [ ]:
# Load SFT training data
TRAIN_PATH = f"{config['paths']['data_filtered']}/sft_train.json"
VAL_PATH = f"{config['paths']['data_filtered']}/sft_val.json"

with open(TRAIN_PATH, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(VAL_PATH, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

In [ ]:
# Preview data format
print("Sample training data:")
print(json.dumps(train_data[0], indent=2, ensure_ascii=False)[:500])

In [ ]:
# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print(f"Train dataset: {train_dataset}")
print(f"Val dataset: {val_dataset}")

## 3. Load Base Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Choose your base model
# Options: meta-llama/Llama-3.2-3B, mistralai/Mistral-7B-v0.1, Qwen/Qwen2.5-3B
BASE_MODEL_ID = config['models']['sft_base']

print(f"Loading base model: {BASE_MODEL_ID}")

In [ ]:
# 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("Quantization config created")

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# Set pad token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.padding_side = "right"  # Important for training

print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")
print(f"PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

In [ ]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Enable gradient checkpointing for memory efficiency
model.config.use_cache = False
model.config.pretraining_tp = 1

# Log memory after model load
mem = tracker.log_memory()
print(f"Model loaded!")
print(f"GPU Memory: {mem['allocated']:.2f} GB allocated, {mem['reserved']:.2f} GB reserved")

## 4. Configure LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

print("Model prepared for k-bit training")

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=config['training']['lora_r'],
    lora_alpha=config['training']['lora_alpha'],
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=config['training']['lora_dropout'],
    bias="none",
    task_type="CAUSAL_LM",
)

print("LoRA config:")
print(f"  r: {lora_config.r}")
print(f"  alpha: {lora_config.lora_alpha}")
print(f"  dropout: {lora_config.lora_dropout}")
print(f"  target_modules: {lora_config.target_modules}")

In [ ]:
# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Log parameters for efficiency comparison
tracker.log_params(model)

# Print trainable parameters
print(f"\nTrainable parameters: {tracker.metrics['trainable_params']:,} ({tracker.metrics['trainable_ratio']*100:.2f}%)")
print(f"Total parameters: {tracker.metrics['total_params']:,}")

## 5. Format Training Data

In [ ]:
def format_instruction(sample):
    """
    Format instruction-response pair into a prompt for training.
    Uses Llama 3 chat template format.
    """
    instruction = sample["instruction"]
    response = sample["output"]
    
    # Llama 3 format
    text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{response}<|eot_id|>"
    
    return {"text": text}

# Test formatting
sample_formatted = format_instruction(train_data[0])
print("Sample formatted text:")
print(sample_formatted["text"][:500])

In [ ]:
# Apply formatting to datasets
train_dataset = train_dataset.map(format_instruction, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(format_instruction, remove_columns=val_dataset.column_names)

print(f"Formatted train dataset: {train_dataset}")
print(f"Formatted val dataset: {val_dataset}")

## 6. Configure Training

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Output directory
OUTPUT_DIR = f"{config['paths']['models_sft']}/sft-checkpoint"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Training hyperparameters
    num_train_epochs=config['training']['sft_epochs'],
    per_device_train_batch_size=config['training']['sft_batch_size'],
    per_device_eval_batch_size=config['training']['sft_batch_size'],
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
    
    # Optimizer
    learning_rate=config['training']['sft_learning_rate'],
    weight_decay=0.01,
    optim="paged_adamw_32bit",
    
    # Learning rate schedule
    lr_scheduler_type="cosine",
    warmup_steps=100,
    
    # Logging
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=10,
    logging_first_step=True,
    
    # Evaluation
    evaluation_strategy="steps",
    eval_steps=200,
    
    # Checkpointing
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    
    # Performance
    fp16=True,
    gradient_checkpointing=True,
    
    # Misc
    report_to="none",  # Disable wandb/tensorboard
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

print("Training arguments:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Total steps: ~{len(train_dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

## 7. Initialize Trainer

In [ ]:
# SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

print("SFT Trainer initialized!")

## 8. Train Model

In [ ]:
# Start training with timing
print("Starting SFT training...")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

tracker.start_training()
train_result = trainer.train()
tracker.end_training()

print("\n" + "=" * 50)
print(f"Training completed!")
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Training time: {tracker.metrics['training_time_seconds']/3600:.2f} hours")

In [ ]:
# Print training metrics
print("\nTraining metrics:")
print(f"  Final train loss: {train_result.training_loss:.4f}")
print(f"  Total steps: {train_result.global_step}")

## 9. Evaluate Model

In [ ]:
# Evaluate on validation set
print("Evaluating model...")
eval_results = trainer.evaluate()

print("\nEvaluation metrics:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

## 10. Test Generation

In [ ]:
def generate_response(instruction: str, max_new_tokens: int = 256):
    """Generate a response for the given instruction."""
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        start_time = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        gen_time = time.time() - start_time
    
    tokens_generated = outputs.shape[1] - inputs['input_ids'].shape[1]
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Extract response
    if "<|start_header_id|>assistant<|end_header_id|>" in generated:
        response = generated.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        response = response.split("<|eot_id|>")[0].strip()
        return response, tokens_generated, gen_time
    
    return generated, tokens_generated, gen_time

print("Generation function defined!")

In [ ]:
# Test generation and measure inference speed
test_instructions = [
    "Explain the concept of machine learning in simple terms.",
    "Write a Python function to calculate the factorial of a number.",
    "What are the main differences between supervised and unsupervised learning?",
]

print("Testing fine-tuned model generation:")
print("=" * 50)

total_tokens = 0
total_time = 0

for i, instruction in enumerate(test_instructions):
    print(f"\n[Test {i+1}]")
    print(f"Instruction: {instruction}")
    print(f"\nResponse:")
    response, tokens, gen_time = generate_response(instruction, max_new_tokens=200)
    print(response)
    print(f"\nTokens: {tokens}, Time: {gen_time:.2f}s, Speed: {tokens/gen_time:.1f} tok/s")
    print("-" * 50)
    
    total_tokens += tokens
    total_time += gen_time

# Log inference speed
tracker.log_inference_speed(total_tokens, total_time)
print(f"\nAverage inference speed: {tracker.metrics['inference_tokens_per_sec']:.1f} tokens/sec")

## 11. Save Fine-tuned Model

In [ ]:
# Save the final model
FINAL_MODEL_DIR = f"{config['paths']['models_sft']}/final"

print(f"Saving final model to: {FINAL_MODEL_DIR}")

trainer.save_model(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print("Model saved!")

In [ ]:
# Save training configuration
training_config = {
    "base_model": BASE_MODEL_ID,
    "training_data_size": len(train_data),
    "validation_data_size": len(val_data),
    "lora_config": {
        "r": lora_config.r,
        "alpha": lora_config.lora_alpha,
        "dropout": lora_config.lora_dropout,
        "target_modules": lora_config.target_modules,
    },
    "training_args": {
        "epochs": training_args.num_train_epochs,
        "batch_size": training_args.per_device_train_batch_size,
        "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
        "learning_rate": training_args.learning_rate,
    },
    "results": {
        "train_loss": train_result.training_loss,
        "eval_loss": eval_results["eval_loss"],
        "total_steps": train_result.global_step,
    },
    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
}

config_path = f"{FINAL_MODEL_DIR}/training_config.json"
with open(config_path, 'w') as f:
    json.dump(training_config, f, indent=2)

print(f"Training config saved to: {config_path}")

## 12. Plot Training Curves

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load training logs
log_history = trainer.state.log_history

# Extract train and eval losses
train_logs = [log for log in log_history if 'loss' in log]
eval_logs = [log for log in log_history if 'eval_loss' in log]

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training loss
if train_logs:
    steps = [log['step'] for log in train_logs]
    losses = [log['loss'] for log in train_logs]
    axes[0].plot(steps, losses, label='Train Loss')
    axes[0].set_xlabel('Steps')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# Evaluation loss
if eval_logs:
    steps = [log['step'] for log in eval_logs]
    losses = [log['eval_loss'] for log in eval_logs]
    axes[1].plot(steps, losses, label='Eval Loss', color='orange')
    axes[1].set_xlabel('Steps')
    axes[1].set_ylabel('Loss')
    axes[1].set_title('Evaluation Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{config['paths']['evaluation_figures']}/sft_training_curves.png", dpi=150)
plt.show()

print(f"Training curves saved to {config['paths']['evaluation_figures']}/sft_training_curves.png")

In [ ]:
# Save efficiency metrics for comparison
METRICS_DIR = f"{PROJECT_ROOT}/evaluation/metrics"
os.makedirs(METRICS_DIR, exist_ok=True)

# Get summary
summary = tracker.get_summary()

# Add training results
summary["train_loss"] = train_result.training_loss
summary["eval_loss"] = eval_results["eval_loss"]

# Save
tracker.save(f"{METRICS_DIR}/lora_metrics.json")

print("\n=== LoRA Efficiency Metrics Summary ===")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

## 13. Cleanup

In [ ]:
# Free GPU memory
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

print("Memory cleared!")

## ✓ SFT Training Complete!

### Summary:
- Fine-tuned model saved to `models/sft/final/`
- LoRA adapters trained with parameter-efficient fine-tuning
- Training configuration and metrics saved

### Model Loading:
To load the fine-tuned model later:
```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID)
model = PeftModel.from_pretrained(base_model, FINAL_MODEL_DIR)
tokenizer = AutoTokenizer.from_pretrained(FINAL_MODEL_DIR)
```

### Next Steps:
1. Proceed to `06_dpo_training.ipynb` for DPO alignment
2. This will further improve the model using preference data